In [ ]:
import torch
from transformers import BertTokenizer, BertForQuestionAnswering

In [ ]:
# Load the pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
def answer_question(question, context):
    # Tokenize the input question and context
    encoded_input = tokenizer.encode_plus(question, context, return_tensors='pt')
    
    # Retrieve the token IDs, attention mask, and token type IDs
    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)
    token_type_ids = encoded_input['token_type_ids'].to(device)
    
    # Use the BERT model to get the start and end logits for the answer
    start_logits, end_logits = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
    
    # Find the start and end indices of the answer in the input context
    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits) + 1
    
    # Decode the answer from the token IDs
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][start_index:end_index]))
    
    return answer

In [ ]:
while True:
    user_input = input("User: ")
    if user_input.lower() == 'exit':
        print("Chatbot: Goodbye!")
        break
    
    context = user_input
    question = input("User: ")
    
    answer = answer_question(question, context)
    print("Chatbot:", answer)
